In [15]:
import json
path='./data/example/real.txt'
txt = open(path, 'r')
lines=[]
for line in txt:
    line = line.strip('\n').rstrip()
    lines.append(line)
lines


['书：请公诉人入庭。',
 '书：法庭肃静，现在宣布法庭纪律。',
 '根据《中华人民共和国人民法院法庭规则》的规定，法庭审理过程中，全体人员在庭审活动中应当服从审判长的指挥，尊重司法礼仪，遵守法庭纪律:',
 '1.不得鼓掌、喧哗；',
 '2.不得吸烟、进食；',
 '3.未经允许不得对庭审活动进行录音、录像、拍照或使用移动通信工具等传播庭审活动；经许可录音、录像、拍照的媒体记者，应当在指定的时间和区域进行，不得影响或干扰庭审活动；',
 '4.不得实施其他危害法庭安全或妨害法庭秩序的行为。',
 '检察人员、诉讼参与人发言或提问，应当经审判长许可。',
 '旁听人员不得进入审判活动区，不得随意站立、走动，不得发言、提问。',
 '违反法庭规则对庭审活动进行录音、录像、拍照或使用移动通信工具等传播庭审活动，人民法院可以暂扣其使用的设备及存储介质，删除相关内容。',
 '对违反法庭纪律的人员，审判长将警告制止、予以训诫、或者指令法警强行带出法庭。危及法庭安全或扰乱法庭秩序的，处以罚款、拘留；构成犯罪的，依法追究刑事责任。',
 '书：全体起立！',
 '书：请审判员入庭。',
 '审：全体坐下！',
 '书：报告审判长，开庭前的准备工作已经就绪，可以开庭！',
 '审：XXX人民法院现在开庭！传被告人张XX到庭。',
 '审：请执勤法警给被告人打开戒俱。',
 '审：被告人张XX，根据案卷材料记载，你是198X年X月13日出生于山东省XXXX，男，汉族，初中文化，公民身份证号码XXXXXXXXXXXXXXXX，户籍地山东省XXXX，因殴打他人，于2022年7月11日被拘留三日。因涉嫌犯有故意伤害罪，于2023年6月16日被刑事拘留，同年6月19日取保候审。被告人，刚才所宣读的你的身份情况属实吗？',
 '被：属实。',
 '审：你是否是中共党员、人大代表、政协委员或国家公职人员？或其他政治身份？',
 '被：不是。',
 '审：被告人张XX，你历史上是否受过其他法律处分？',
 '被：去年有个治安处罚，除了这个没有了。',
 '审：XXX人民检察院的起诉书、我院的诉讼权利义务告知书，被告人张XX收到没有？',
 '被：收到了。',
 '审：什么时间收到的？',
 '被：2023年X月25日。',
 '审：被告人张XX，你的案件已经由XXX人民检察院提起公诉，XX

In [3]:
names=['被','审','书','公']
output_dict={}
# dict.update({"1":5})
dialog=""
name=""
cnt=0
for line in lines:
    if len(line)==0:
        continue
    tmp=line.split("：")
    ch=tmp[0][0]
    # print(tmp[0])
    if len(tmp[0])==1 and ch in names:
        if len(dialog)!=0:
            cnt+=1
            output_dict.update({cnt:{"person":name,"sentence":dialog,"goal":0}})
        dialog=line[2:]
        name=ch
    else:
        dialog+=line
if len(dialog)!=0:
    cnt+=1
    output_dict.update({cnt:{"person":name,"sentence":dialog,"goal":0}})
output_dict

NameError: name 'lines' is not defined

In [17]:
with open('./data/example/real.json', 'w') as file:
    json.dump(output_dict,file,ensure_ascii=False)

In [6]:
import sys
sys.path.append("..") 

import uuid
import logging
import argparse
import re
import json
import jieba
import math
import os
import datetime
import time
import collections
from agent import Agent
from typing import List, Dict, Any, Tuple
from LLM.deli_client import search_law
from LLM.apillm import APILLM
from rouge_chinese import Rouge

In [7]:
import openai

client = openai.OpenAI(
    base_url="https://svip.xty.app/v1",
    api_key="sk-r0WeYOdkMjzYdnSxEcC8B931Aa904e4bBaCcAc2a57D803F1"
)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",  # 指定使用的大模型是谁，gpt-3.5-turbo 是chatgpt3.5，
  messages=[  # 提示词，
    {"role": "system", "content": "你是一个老师."},  # 让chatgpt充当什么身份
    {"role": "user", "content": "编写一段歌颂元宵节的歌词"}, # 具体要chatgpt完成的操作了。
    # {"role": "assistant", "content": "在这元宵佳节临，灯火闪烁映星河。\n汇聚亲情团圆欢，共享团圆喜庆多。\n饮甜汤圆舒心意，挂灯花灯亮明朗。\n传统民俗传千古，文化传承爱不忘。\n元宵圆圆思亲亲，喜庆团圆共欢颜。"},  # 约束条件或基于什么条件
  ]
)

print(completion.choices[0].message)


ChatCompletionMessage(content='**歌颂元宵节的歌词**\n\n**副歌：**  \n灯火辉煌照九天，  \n元宵节里笑声甜，  \n圆圆汤圆寓吉祥，  \n团圆欢乐满人间。  \n共赏明月共此时，  \n一片心愿寄天涯，  \n元宵夜，心连心，  \n幸福伴随每一步。\n\n**第一段：**  \n花灯如梦照街头，  \n火树银花绽芳愁，  \n千家万户迎春到，  \n彩灯照亮希望道。  \n烟花舞动夜空明，  \n笑语欢歌声声传，  \n手提灯笼走一程，  \n情深意浓满心田。\n\n**副歌：**  \n灯火辉煌照九天，  \n元宵节里笑声甜，  \n圆圆汤圆寓吉祥，  \n团圆欢乐满人间。  \n共赏明月共此时，  \n一片心愿寄天涯，  \n元宵夜，心连心，  \n幸福伴随每一步。\n\n**第二段：**  \n汤圆滚滚情意浓，  \n甜蜜滋味暖心胸，  \n无论远近齐团圆，  \n手牵手心连心愿。  \n年年岁岁元宵节，  \n幸福如灯长常明，  \n愿这温暖的夜晚，  \n照亮一生的美梦。  \n\n**副歌：**  \n灯火辉煌照九天，  \n元宵节里笑声甜，  \n圆圆汤圆寓吉祥，  \n团圆欢乐满人间。  \n共赏明月共此时，  \n一片心愿寄天涯，  \n元宵夜，心连心，  \n幸福伴随每一步。\n\n这段歌词以元宵节的灯火、汤圆、团圆等元素为主题，歌颂了节日的欢乐和象征的美好寓意。希望你喜欢！', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [12]:
api_pool = {
    "o1-mini": 1,
    "gpt-4": 2,
    "gpt-4o-mini": 3,
    "gpt-3.5-turbo": 4,
    "claude-3-sonnet": 5,
    "legalone": 6,
    "chatglm-4-air": 7
}
list(api_pool.keys())

['o1-mini',
 'gpt-4',
 'gpt-4o-mini',
 'gpt-3.5-turbo',
 'claude-3-sonnet',
 'legalone',
 'chatglm-4-air']